In [16]:
import pandas as pd
import numpy as np
import re
from nltk.stem import LancasterStemmer
import dill

In [17]:
df = pd.read_csv('../DataFolder/Original_CSV/yrs12_passages.csv')

In [18]:
passage = df['passage'].values

In [19]:
unique_word = set()
stem = LancasterStemmer()
for e in passage:
    #match all word using regex and add to set
    word_ls = list(map(lambda x: stem.stem(x), re.findall(r'\w+', e.lower())))
    unique_word.update(word_ls)

In [20]:
#Using dill to save the set
with open('unique_word.dill', 'wb') as f:
    dill.dump(unique_word, f)

In [13]:
len(unique_word)

1674

In [ ]:
# TF2 version
import tensorflow as tf
import tensorflow_hub as hub

m = hub.KerasLayer('https://tfhub.dev/google/trillsson5/1')
# NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
# the shape [batch size, time].

In [38]:
audio_samples = tf.random.normal([3, 64000])
embeddings = m(audio_samples)['embedding']
# Models internally aggregate over time. For a time-series of embeddings, the
# user can frame audio however they want.
embeddings.shape.assert_is_compatible_with([None, 1024])

In [34]:
embeddings

<tf.Tensor: shape=(3, 1024), dtype=float32, numpy=
array([[-1.5436294 ,  0.3506941 , -0.48050883, ...,  2.8203986 ,
        -0.82041126, -1.9124141 ],
       [-1.5482461 ,  0.3428351 , -0.4758531 , ...,  2.824588  ,
        -0.83504   , -1.9144915 ],
       [-1.5582081 ,  0.3560771 , -0.48131317, ...,  2.794648  ,
        -0.83085126, -1.917218  ]], dtype=float32)>

In [39]:
embeddings

<tf.Tensor: shape=(3, 1024), dtype=float32, numpy=
array([[-1.1291425 ,  0.4191693 ,  0.87145674, ...,  0.97673386,
        -1.1060619 , -1.4092574 ],
       [-1.1275656 ,  0.44298258,  0.8755819 , ...,  0.92630476,
        -1.0676109 , -1.4421777 ],
       [-1.1662369 ,  0.46046028,  0.8969983 , ...,  0.9104195 ,
        -1.083088  , -1.4256439 ]], dtype=float32)>

In [40]:
model = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2-960h/1")
# For using this model, it's important to set `jit_compile=True` on GPUs/CPUs
# as some operations in this model (i.e. group-convolutions) are unsupported without it
model = tf.function(model, jit_compile=True)